In [1]:
from predict import PredictWNet
from config import Config
import tifffile as tiff
import numpy as np
import sys
import napari

sys.path.append("../..")
from utils import create_dataset_dict_no_labs

C:\Users\Cyril\anaconda3\envs\cellseg3d\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = Config()
predictor = PredictWNet(
    trained_model_path=r"new_results_wnet_1500e/test_wnet_1500e.pth",
    config=config,
    crf=False,
)

train_files = create_dataset_dict_no_labs(
    volume_directory=r"../../dataset/cropped_visual/val/volumes"
)
train_files = [d.get("image") for d in train_files]
volumes = tiff.imread(train_files).astype(np.float32)

In [3]:
segmentations = predictor.predict_batch(volumes)
#segmentations = np.where(segmentations > 0.5, 0, 1)
tiff.imwrite(r"../../results/segmentations.tif", segmentations)

In [10]:
segmentations_max = np.argmax(segmentations, axis=1)
segmentation_0 = np.where(segmentations_max == 0, segmentations_max + 1, 0)
segmentation_1 = np.where(segmentations_max == 1, segmentations_max, 0)
segmentation_2 = np.where(segmentations_max == 2, segmentations_max - 1, 0)
print(segmentation_0.shape)

(7, 64, 64, 64)


In [15]:
viewer = napari.viewer.Viewer()
for i in range(segmentation_0.shape[0]):
    viewer.add_image(volumes[i], name=f"Original image {i}", colormap="inferno")
    # viewer.add_labels(segmentation_0[i], name=f"Segmentation 0 image {i}")
    viewer.add_labels(segmentation_1[i], name=f"Segmentation 1 image {i}")
    # viewer.add_labels(segmentation_2[i], name=f"Segmentation 2 image {i}")

In [16]:
napari.run()